In [46]:
import numpy as np
import pandas as pd

import torch

In [47]:
from pts.dataset import to_pandas, MultivariateGrouper, TrainDatasets
from pts.dataset.repository import get_dataset, dataset_recipes
from pts.model.tempflow import TempFlowEstimator
from pts.model.transformer_tempflow import TransformerTempFlowEstimator
from pts import Trainer
from pts.evaluation import make_evaluation_predictions
from pts.evaluation import MultivariateEvaluator

In [48]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Prepeare data set

In [49]:
dataset = get_dataset("solar_nips", regenerate=False, shuffle=False) # 得到三个文件train/data.json、test/data.json以及metadata.json

2021-02-27 21:18:23,008 - INFO - using dataset already processed in path /home/yaoqf/.pytorch/pytorch-ts/datasets/solar_nips.


In [50]:
dataset.metadata

MetaData(freq='H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='137')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=24)

In [51]:
train_grouper = MultivariateGrouper(max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))

test_grouper = MultivariateGrouper(num_test_dates=int(len(dataset.test)/len(dataset.train)), 
                                   max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))

In [52]:
dataset_train = train_grouper(dataset.train)
dataset_test = test_grouper(dataset.test)

2021-02-27 21:18:32,648 - INFO - first/last timestamp found: 2006-01-01 00:00:00/2006-10-20 00:00:00
2021-02-27 21:18:32,649 - INFO - group training time-series to datasets
2021-02-27 21:18:34,044 - INFO - first/last timestamp found: 2006-01-01 00:00:00/2006-10-27 00:00:00
2021-02-27 21:18:34,048 - INFO - group test time-series to datasets


## Evaluator

In [53]:
evaluator = MultivariateEvaluator(quantiles=(np.arange(20)/20.0)[1:],
                                  target_agg_funcs={'sum': np.sum})

## `GRU-MAF`

In [62]:
estimator = TempFlowEstimator(
    target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
    prediction_length=dataset.metadata.prediction_length,
    cell_type='GRU',
    input_size=552,
    freq=dataset.metadata.freq,
    scaling=True,
    dequantize=True,
    flow_type='MAF',
    trainer=Trainer(device=device,
                    epochs=25,
                    learning_rate=1e-3,
                    num_batches_per_epoch=100,
                    batch_size=64)
)

In [63]:
predictor = estimator.train(dataset_train)
forecast_it, ts_it = make_evaluation_predictions(dataset=dataset_test,
                                             predictor=predictor,
                                             num_samples=100)
forecasts = list(forecast_it)
targets = list(ts_it)

agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))

99it [00:10,  9.71it/s, avg_epoch_loss=-26.2, epoch=0]
99it [00:11,  8.92it/s, avg_epoch_loss=-146, epoch=1]
99it [00:11,  8.95it/s, avg_epoch_loss=-165, epoch=2]
99it [00:10,  9.31it/s, avg_epoch_loss=-178, epoch=3]
99it [00:10,  9.29it/s, avg_epoch_loss=-185, epoch=4]
99it [00:10,  9.19it/s, avg_epoch_loss=-190, epoch=5]
99it [00:10,  9.12it/s, avg_epoch_loss=-194, epoch=6]
99it [00:10,  9.12it/s, avg_epoch_loss=-197, epoch=7]
99it [00:10,  9.16it/s, avg_epoch_loss=-200, epoch=8]
99it [00:10,  9.26it/s, avg_epoch_loss=-203, epoch=9]
99it [00:10,  9.45it/s, avg_epoch_loss=-205, epoch=10]
99it [00:10,  9.20it/s, avg_epoch_loss=-207, epoch=11]
99it [00:10,  9.22it/s, avg_epoch_loss=-208, epoch=12]
99it [00:10,  9.20it/s, avg_epoch_loss=-209, epoch=13]
99it [00:10,  9.16it/s, avg_epoch_loss=-211, epoch=14]
99it [00:10,  9.28it/s, avg_epoch_loss=-212, epoch=15]
99it [00:10,  9.61it/s, avg_epoch_loss=-213, epoch=16]
99it [00:10,  9.35it/s, avg_epoch_loss=-214, epoch=17]
99it [00:10,  9.18i

### Metrics

In [64]:
print("CRPS: {}".format(agg_metric['mean_wQuantileLoss']))
print("ND: {}".format(agg_metric['ND']))
print("NRMSE: {}".format(agg_metric['NRMSE']))
print("MSE: {}".format(agg_metric['MSE']))

CRPS: 0.38148718058575826
ND: 0.48739644950924027
NRMSE: 1.0032379403994989
MSE: 954.7412421352797


In [65]:
print("CRPS-Sum: {}".format(agg_metric['m_sum_mean_wQuantileLoss']))
print("ND-Sum: {}".format(agg_metric['m_sum_ND']))
print("NRMSE-Sum: {}".format(agg_metric['m_sum_NRMSE']))
print("MSE-Sum: {}".format(agg_metric['m_sum_MSE']))

CRPS-Sum: 0.32853644393810727
ND-Sum: 0.421502668453047
NRMSE-Sum: 0.7682724166824968
MSE-Sum: 10508710.428571431


## `GRU-Real-NVP`

In [66]:
estimator = TempFlowEstimator(
    target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
    prediction_length=dataset.metadata.prediction_length,
    cell_type='GRU',
    input_size=552,
    freq=dataset.metadata.freq,
    scaling=True,
    dequantize=True,
    n_blocks=4,
    trainer=Trainer(device=device,
                    epochs=45,
                    learning_rate=1e-3,
                    num_batches_per_epoch=100,
                    batch_size=64)
)

In [67]:
predictor = estimator.train(dataset_train)
forecast_it, ts_it = make_evaluation_predictions(dataset=dataset_test,
                                             predictor=predictor,
                                             num_samples=100)
forecasts = list(forecast_it)
targets = list(ts_it)

agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))

99it [00:11,  8.71it/s, avg_epoch_loss=-54.9, epoch=0]
99it [00:10,  9.22it/s, avg_epoch_loss=-138, epoch=1]
99it [00:10,  9.08it/s, avg_epoch_loss=-153, epoch=2]
99it [00:11,  8.61it/s, avg_epoch_loss=-161, epoch=3]
99it [00:11,  8.84it/s, avg_epoch_loss=-166, epoch=4]
99it [00:11,  8.76it/s, avg_epoch_loss=-171, epoch=5]
99it [00:10,  9.34it/s, avg_epoch_loss=-173, epoch=6]
99it [00:10,  9.17it/s, avg_epoch_loss=-176, epoch=7]
99it [00:11,  8.96it/s, avg_epoch_loss=-179, epoch=8]
99it [00:10,  9.08it/s, avg_epoch_loss=-182, epoch=9]
99it [00:11,  8.87it/s, avg_epoch_loss=-183, epoch=10]
99it [00:11,  8.60it/s, avg_epoch_loss=-185, epoch=11]
99it [00:10,  9.37it/s, avg_epoch_loss=-187, epoch=12]
99it [00:10,  9.13it/s, avg_epoch_loss=-189, epoch=13]
99it [00:12,  8.17it/s, avg_epoch_loss=-190, epoch=14]
99it [00:11,  8.68it/s, avg_epoch_loss=-191, epoch=15]
99it [00:10,  9.18it/s, avg_epoch_loss=-193, epoch=16]
99it [00:11,  8.80it/s, avg_epoch_loss=-194, epoch=17]
99it [00:11,  8.77i

### Metrics

In [68]:
print("CRPS: {}".format(agg_metric['mean_wQuantileLoss']))
print("ND: {}".format(agg_metric['ND']))
print("NRMSE: {}".format(agg_metric['NRMSE']))
print("MSE: {}".format(agg_metric['MSE']))

CRPS: 0.40106700392773786
ND: 0.5202773674186033
NRMSE: 1.0665755849085081
MSE: 1079.0984367334806


In [69]:
print("CRPS-Sum: {}".format(agg_metric['m_sum_mean_wQuantileLoss']))
print("ND-Sum: {}".format(agg_metric['m_sum_ND']))
print("NRMSE-Sum: {}".format(agg_metric['m_sum_NRMSE']))
print("MSE-Sum: {}".format(agg_metric['m_sum_MSE']))

CRPS-Sum: 0.36216298231000654
ND-Sum: 0.45842982523421666
NRMSE-Sum: 0.8217251271877606
MSE-Sum: 12021871.392857144


## `Transformer-MAF`

In [70]:
estimator = TransformerTempFlowEstimator(
    d_model=16,
    num_heads=4,
    input_size=552,
    target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
    prediction_length=dataset.metadata.prediction_length,
    context_length=dataset.metadata.prediction_length*4,
    flow_type='MAF',
    dequantize=True,
    freq=dataset.metadata.freq,
    trainer=Trainer(
        device=device,
        epochs=14,
        learning_rate=1e-3,
        num_batches_per_epoch=100,
        batch_size=64,
    )
)

In [71]:
predictor = estimator.train(dataset_train)
forecast_it, ts_it = make_evaluation_predictions(dataset=dataset_test,
                                             predictor=predictor,
                                             num_samples=100)
forecasts = list(forecast_it)
targets = list(ts_it)

agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))

99it [00:11,  8.56it/s, avg_epoch_loss=-43.4, epoch=0]
99it [00:11,  8.93it/s, avg_epoch_loss=-162, epoch=1]
99it [00:11,  8.94it/s, avg_epoch_loss=-181, epoch=2]
99it [00:10,  9.15it/s, avg_epoch_loss=-195, epoch=3]
99it [00:11,  8.43it/s, avg_epoch_loss=-203, epoch=4]
99it [00:11,  8.91it/s, avg_epoch_loss=-209, epoch=5]
99it [00:11,  8.61it/s, avg_epoch_loss=-213, epoch=6]
99it [00:11,  8.84it/s, avg_epoch_loss=-216, epoch=7]
99it [00:11,  8.82it/s, avg_epoch_loss=-218, epoch=8]
99it [00:11,  8.86it/s, avg_epoch_loss=-220, epoch=9]
99it [00:10,  9.04it/s, avg_epoch_loss=-221, epoch=10]
99it [00:11,  8.83it/s, avg_epoch_loss=-223, epoch=11]
99it [00:10,  9.12it/s, avg_epoch_loss=-224, epoch=12]
99it [00:11,  8.80it/s, avg_epoch_loss=-225, epoch=13]
Running evaluation: 7it [00:01,  3.94it/s]
Running evaluation: 7it [00:01,  3.98it/s]
Running evaluation: 7it [00:01,  3.97it/s]
Running evaluation: 7it [00:01,  3.94it/s]
Running evaluation: 7it [00:01,  3.99it/s]
Running evaluation: 7it 

### Metrics

In [72]:
print("CRPS: {}".format(agg_metric['mean_wQuantileLoss']))
print("ND: {}".format(agg_metric['ND']))
print("NRMSE: {}".format(agg_metric['NRMSE']))
print("MSE: {}".format(agg_metric['MSE']))

CRPS: 0.3973669545116235
ND: 0.5272711915441427
NRMSE: 1.0469494137161859
MSE: 1039.7506092003248


In [73]:
print("CRPS-Sum: {}".format(agg_metric['m_sum_mean_wQuantileLoss']))
print("ND-Sum: {}".format(agg_metric['m_sum_ND']))
print("NRMSE-Sum: {}".format(agg_metric['m_sum_NRMSE']))
print("MSE-Sum: {}".format(agg_metric['m_sum_MSE']))

CRPS-Sum: 0.3390463815731258
ND-Sum: 0.4433532171953211
NRMSE-Sum: 0.8165239227580265
MSE-Sum: 11870165.380952382
